## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [10]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [11]:
ct("List",7,2)
exe("""
select 費目,出金額 from 家計簿
where 出金額=(select max(出金額) from 家計簿)
""")

[('水道光熱費', 7560)]

In [12]:
ct("List",7,3)
exe("""
update 家計簿集計
set 平均=(select avg(出金額)
from 家計簿アーカイブ
where 出金額>0
and 費目='食費'
)
where 費目='食費'
""")
exe("select * from 家計簿集計")

[('居住費', 160000, 80000, 80000, 80000, 3),
 ('水道光熱費', 11760, 5880, 7560, 4200, 2),
 ('教養娯楽費', 4600, 2300, 2800, 1800, 2),
 ('給料', 840000, 280000, 280000, 280000, 3),
 ('食費', 10380, 5000, 5000, 380, 3)]

In [13]:
ct("List",7,4)
exe("""
select 日付,(select 合計 from 家計簿集計 where 費目='食費')
from 家計簿アーカイブ
""")

[(datetime.date(2017, 12, 10), 10380),
 (datetime.date(2017, 12, 18), 10380),
 (datetime.date(2017, 12, 24), 10380),
 (datetime.date(2017, 12, 25), 10380),
 (datetime.date(2018, 1, 10), 10380),
 (datetime.date(2018, 1, 13), 10380),
 (datetime.date(2018, 1, 13), 10380),
 (datetime.date(2018, 1, 25), 10380)]

In [14]:
ct("List",7,5)
exe("""
select * from 家計簿集計
where 費目 in ('食費','水道光熱費')
""")

[('水道光熱費', 11760, 5880, 7560, 4200, 2), ('食費', 10380, 3460, 5000, 380, 3)]

In [15]:
ct("List",7,6)
exe("""
select * from 家計簿集計
where 費目 in (select distinct 費目 from 家計簿)
""")

[('水道光熱費', 11760, 5880, 7560, 4200, 2),
 ('食費', 10380, 3460, 5000, 380, 3),
 ('教養娯楽費', 4600, 2300, 2800, 1800, 2),
 ('給料', 840000, 280000, 280000, 280000, 3)]

In [16]:
ct("List",7,7)
exe("""
select * from 家計簿
where 費目='食費'
and 出金額<
any(select 出金額 from 家計簿アーカイブ
where 費目='食費')
""")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380)]

In [17]:
ct("List",7,8)
exe("""
select * from 家計簿アーカイブ
where 費目 in (select 費目 from 家計簿
where 費目 is not null)
""")

[(datetime.date(2017, 12, 10), '給料', '11月の給料', 280000, 0),
 (datetime.date(2017, 12, 18), '水道光熱費', '水道代', 0, 4200),
 (datetime.date(2017, 12, 24), '食費', 'レストランみやび', 0, 5000),
 (datetime.date(2018, 1, 10), '給料', '12月の給料', 280000, 0),
 (datetime.date(2018, 1, 13), '教養娯楽費', 'スッキリシネマズ', 0, 1800),
 (datetime.date(2018, 1, 13), '食費', '新年会', 0, 5000)]

In [18]:
ct("List",7,11)
exe("""
select sum(sub.出金額) as 出金額合計
from 
(select 日付,費目,出金額
from 家計簿
union
select 日付,費目,出金額
from 家計簿アーカイブ
where 日付>='2018-01-01'
and
日付<='2018-01-31') as sub
""")

[(102540,)]

In [19]:
ct("List",7,12)
exe("""
insert into 家計簿集計 (費目,合計,平均,回数)
select 費目,sum(出金額),avg(出金額),0
from 家計簿 where 出金額>0
group by 費目
""")
exe_df("select * from 家計簿集計")

,費目,合計,平均,最大,最小,回数
0,食費,5580,2790,None,None,0
1,交際費,5000,5000,None,None,0
2,教養娯楽費,2800,2800,None,None,0
3,水道光熱費,7560,7560,None,None,0


## Q

In [20]:
ct("Q",7,3,1)
exe("""
insert into 頭数集計 (飼育県,頭数)
select 飼育県,count(*) from 個体識別
group by 飼育県
""")
exe("select * from 頭数集計")

[('千葉県', 3), ('群馬県', 2), ('岩手県', 1), ('北海道', 4)]

In [21]:
ct("Q",7,3,2)
exe("""
select 飼育県,個体識別番号,雌雄コード,
case 雌雄コード when '1' then '雄'
else  '雌'
end
from 個体識別
where 飼育県 in
(select 飼育県 from 頭数集計
order by 頭数 desc
offset 0 rows fetch next 3 rows only)
""")

[('千葉県', '1211', '2', '雌'),
 ('群馬県', '4530', '2', '雌'),
 ('千葉県', '2237', '1', '雄'),
 ('北海道', '8105', '1', '雄'),
 ('北海道', '1097', '2', '雌'),
 ('北海道', '5014', '1', '雄'),
 ('北海道', '5513', '2', '雌'),
 ('千葉県', '1164', '1', '雄'),
 ('群馬県', '1097', '1', '雄')]